In [1]:
%matplotlib ipympl

In [2]:
# initialization
from rayoptics.environment import *
from rayoptics.raytr import trace

In [3]:
from numpy.linalg import norm
from scipy.optimize import minimize_scalar, bracket, bisect, fsolve  # , newton

In [4]:
root_pth = Path(rayoptics.__file__).resolve().parent

In [5]:
app = AppManager(None)

# Create a new model

In [6]:
app.model = open_model(root_pth/"codev/tests/ag_dblgauss.seq")
opm = app.model
sm = opm.seq_model
osp = opm.optical_spec
pm = opm.parax_model

In [7]:
opm.update_model()

In [8]:
sm.list_model()

             c            t        medium     mode   zdr      sd
 Obj:     0.000000  9.93938e+11       air   transmit  1  2.4782e+11
   1:     0.017793      8.75000    N-SSK2   transmit  1      28.172
   2:     0.006567     0.500000       air   transmit  1      27.136
   3:     0.026537      12.5000     N-SK2   transmit  1      24.626
   4:     0.000000      3.80000        F5   transmit  1      22.894
   5:     0.041269      16.3694       air   transmit  1      17.293
Stop:     0.000000      13.7480       air   transmit  1      15.758
   7:    -0.035239      3.80000        F5   transmit  1      15.135
   8:     0.000000      11.0000    N-SK16   transmit  1      17.250
   9:    -0.026368     0.500000       air   transmit  1      18.926
  10:     0.005637      7.00000    N-SK16   transmit  1      20.349
  11:    -0.012593      61.0873       air   transmit  1      20.776
 Img:     0.000000      0.00000             transmit  1      24.592


In [9]:
stop = sm.stop_surface

In [10]:
def y_stop_coordinate(y1, *args):
    seq_model, ifcx, pt0, dist, wvl, target = args
    pt1 = np.array([0., y1, dist])
    dir0 = pt1 - pt0
    length = norm(dir0)
    dir0 = dir0/length
    ray, _, _ = rt.trace(seq_model, pt0, dir0, wvl)
    y_ray = ray[ifcx][mc.p][1]
    print(y1, y_ray)
    return y_ray - target

In [11]:
def surface_coordinate(coord, *args):
    seq_model, ifcx, pt0, dist, wvl, target = args
    pt1 = np.array([coord[0], coord[1], dist])
    dir0 = pt1 - pt0
    length = norm(dir0)
    dir0 = dir0/length
    ray, _, _ = rt.trace(seq_model, pt0, dir0, wvl)
    xy_ray = np.array([ray[ifcx][mc.p][0], ray[ifcx][mc.p][1]])
    print(coord[0], coord[1], xy_ray[0], xy_ray[1])
    return xy_ray - target

In [12]:
# Newton-Raphson method
import warnings
def newton(func, x0, fprime=None, args=(), tol=1.48e-8, maxiter=50,
           fprime2=None):
    """
    Find a zero using the Newton-Raphson or secant method.

    Find a zero of the function `func` given a nearby starting point `x0`.
    The Newton-Raphson method is used if the derivative `fprime` of `func`
    is provided, otherwise the secant method is used.  If the second order
    derivative `fprime2` of `func` is provided, then Halley's method is used.

    Parameters
    ----------
    func : function
        The function whose zero is wanted. It must be a function of a
        single variable of the form f(x,a,b,c...), where a,b,c... are extra
        arguments that can be passed in the `args` parameter.
    x0 : float
        An initial estimate of the zero that should be somewhere near the
        actual zero.
    fprime : function, optional
        The derivative of the function when available and convenient. If it
        is None (default), then the secant method is used.
    args : tuple, optional
        Extra arguments to be used in the function call.
    tol : float, optional
        The allowable error of the zero value.
    maxiter : int, optional
        Maximum number of iterations.
    fprime2 : function, optional
        The second order derivative of the function when available and
        convenient. If it is None (default), then the normal Newton-Raphson
        or the secant method is used. If it is not None, then Halley's method
        is used.

    Returns
    -------
    zero : float
        Estimated location where function is zero.

    See Also
    --------
    brentq, brenth, ridder, bisect
    fsolve : find zeroes in n dimensions.

    Notes
    -----
    The convergence rate of the Newton-Raphson method is quadratic,
    the Halley method is cubic, and the secant method is
    sub-quadratic.  This means that if the function is well behaved
    the actual error in the estimated zero is approximately the square
    (cube for Halley) of the requested tolerance up to roundoff
    error. However, the stopping criterion used here is the step size
    and there is no guarantee that a zero has been found. Consequently
    the result should be verified. Safer algorithms are brentq,
    brenth, ridder, and bisect, but they all require that the root
    first be bracketed in an interval where the function changes
    sign. The brentq algorithm is recommended for general use in one
    dimensional problems when such an interval has been found.

    Examples
    --------

    >>> def f(x):
    ...     return (x**3 - 1)  # only one real root at x = 1
    
    >>> from scipy import optimize

    ``fprime`` not provided, use secant method
    
    >>> root = optimize.newton(f, 1.5)
    >>> root
    1.0000000000000016
    >>> root = optimize.newton(f, 1.5, fprime2=lambda x: 6 * x)
    >>> root
    1.0000000000000016

    Only ``fprime`` provided, use Newton Raphson method
    
    >>> root = optimize.newton(f, 1.5, fprime=lambda x: 3 * x**2)
    >>> root
    1.0
    
    Both ``fprime2`` and ``fprime`` provided, use Halley's method

    >>> root = optimize.newton(f, 1.5, fprime=lambda x: 3 * x**2,
    ...                        fprime2=lambda x: 6 * x)
    >>> root
    1.0

    """
    if tol <= 0:
        raise ValueError("tol too small (%g <= 0)" % tol)
    if maxiter < 1:
        raise ValueError("maxiter must be greater than 0")
    # Multiply by 1.0 to convert to floating point.  We don't use float(x0)
    # so it still works if x0 is complex.
    p0 = 1.0 * x0
    if fprime is not None:
        # Newton-Rapheson method
        for iter in range(maxiter):
            fder = fprime(p0, *args)
            if fder == 0:
                msg = "derivative was zero."
                warnings.warn(msg, RuntimeWarning)
                return p0
            fval = func(p0, *args)
            newton_step = fval / fder
            if fprime2 is None:
                # Newton step
                p = p0 - newton_step
            else:
                fder2 = fprime2(p0, *args)
                # Halley's method
                p = p0 - newton_step / (1.0 - 0.5 * newton_step * fder2 / fder)
            if abs(p - p0) < tol:
                return p
            p0 = p
    else:
        # Secant method
        if x0 >= 0:
            p1 = x0*(1 + 1e-4) + 1e-4
        else:
            p1 = x0*(1 + 1e-4) - 1e-4
        q0 = func(p0, *args)
        q1 = func(p1, *args)
        for iter in range(maxiter):
            if q1 == q0:
                if p1 != p0:
                    msg = "Tolerance of %s reached" % (p1 - p0)
                    warnings.warn(msg, RuntimeWarning)
                return (p1 + p0)/2.0
            else:
                p = p1 - q1*(p1 - p0)/(q1 - q0)
            if abs(p - p1) < tol:
                return p
            p0 = p1
            q0 = q1
            p1 = p
            q1 = func(p1, *args)
    msg = "Failed to converge after %d iterations, value is %s" % (maxiter, p)
    raise RuntimeError(msg)

In [13]:
fld, wvl, foc = osp.lookup_fld_wvl_focus(2)

In [14]:
fod = osp.parax_data.fod
dist = fod.obj_dist + fod.enp_dist
pt0 = osp.obj_coords(fld)

In [15]:
#res = newton(y_stop_coordinate, 0.)
y_target = 0.
start_y = newton(y_stop_coordinate, 0.,
                 args=(sm, stop, pt0, dist, wvl, y_target))

0.0 0.27914407297214433
0.0001 0.2792027515012124
-0.4757175706440057 -0.025988514825844376
-0.4351993994069977 -1.6269280965730414e-05
-0.43517401840900377 3.3193355839460568e-06
-0.435178319276785 3.3193355839460568e-06


In [16]:
#res = fsolve(surface_coordinate, np.array([0., 0.]),
#             args=(sm, sm.stop_surface, sm.central_wavelength(), np.array([0., 0.])))
xy_target = np.array([0., 0.])
start_coords = fsolve(surface_coordinate, np.array([0., 0.]),
                      epsfcn=0.0001*fod.enp_radius,
                      args=(sm, stop, pt0, dist, wvl, xy_target))

0.0 0.0 0.0 0.27914407297214433
0.0 0.0 0.0 0.27914407297214433
0.0 0.0 0.0 0.27914407297214433
0.05 0.0 0.03180949381683528 0.2791428909007874
0.0 0.05 0.0 0.3111944836824786
0.0 -0.43547659263245836 0.0 -0.00019246161366979404
-1.3002102717272686e-24 -0.43517655122234444 -8.275441943969193e-25 3.3193355839460568e-06
4.063157099147664e-26 -0.4351816382245939 2.586075607490341e-26 3.3193355839460568e-06
1.9388818395892715e-10 -0.4348765098122306 1.2340387944300607e-10 0.000199152734965988
-1.2351997581409052e-24 -0.43517655122234444 -7.8616698467707365e-25 3.3193355839460568e-06
-1.3002102717272686e-24 -0.4134177236612272 -8.275260092588012e-25 0.013986859481259528
6.782246266190498e-33 -0.4351817162127429 4.316692956978446e-33 3.3193355839460568e-06
-1.3002102785096256e-24 -0.4352513687164667 -8.275442496583265e-25 -3.585789955188756e-05
6.781704069421181e-33 -0.43518289021920525 4.316347865266414e-33 3.3193355839460568e-06
-2.163410531083727e-24 -0.435195255595875 -1.3769448404042684

In [17]:
start_coords

array([-1.30021027e-24, -4.35176551e-01])

In [18]:
aim_pt = trace.iterate_ray(opm, sm.stop_surface, np.array([0., 0.]), fld, wvl)

In [19]:
print(start_coords, aim_pt, start_coords[1]-aim_pt[1])

[-1.30021027e-24 -4.35176551e-01] [ 0.         -0.43517617] -3.8237945454433486e-07
